### portmy database: stocks table
### stock database: buy, price tables

In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()
conmy = engine.connect()
engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.set_option('display.float_format','{:,.2f}'.format)

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

today = date.today()
week_str = today.strftime('%Y-%U')
today, week_str

(datetime.date(2022, 4, 23), '2022-16')

### Process after last business day, today must be last business day

In [2]:
today = today - timedelta(days=1)
wkago = today - timedelta(days=10)
today, wkago

(datetime.date(2022, 4, 22), datetime.date(2022, 4, 12))

### Restart & Run All Cells

### This process affects only already in port stocks. To highlight price changes in percent.

In [3]:
cols = 'name price_d price_w percent perform mrkt '.split()

In [4]:
format_dict = {
    'qty':'{:,}',    
    'price':'{:.2f}','price_d':'{:.2f}','price_w':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'maxp':'{:.2f}','minp':'{:.2f}','opnp':'{:.2f}',    
    'pct':'{:,.2f}%','percent':'{:,.2f}%',
    
    'pe':'{:.2f}','pbv':'{:.2f}',
    'paid_up':'{:,.2f}','market_cap':'{:,.2f}',
    'daily_volume':'{:,.2f}','beta':'{:,.2f}', 
    'dly_vol':'{:,.2f}', 
}

In [5]:
sql = """
SELECT name, date, price 
FROM buy 
ORDER BY date DESC 
LIMIT 1
"""
df_buy = pd.read_sql(sql, const)
df_buy.style.format(format_dict)

,name,date,price
0,ORI,2022-04-07,11.00


In [6]:
names = df_buy['name']
name = names.to_string(index=False)

sql = '''
SELECT * FROM stocks WHERE name = '%s'
'''
sql = sql % name
print(sql)

df_stocks = pd.read_sql(sql, conmy)
df_stocks.style.format(format_dict) 


SELECT * FROM stocks WHERE name = 'ORI'



,id,name,market,price,max_price,min_price,pe,pbv,paid_up,market_cap,daily_volume,beta,ticker_id,created_at,updated_at
0,339,ORI,SET100 / SETHD / SETTHSI,11.10,12.70,8.05,8.53,1.82,"1,227.06","27,240.76",116.27,1.32,339,2017-07-23 07:25:07.542864,2022-04-23 07:59:17.253854


In [7]:
sql = '''
SELECT * FROM price WHERE name = "%s" ORDER BY date DESC LIMIT 5
'''
sql = sql % name
print(sql)

df_price = pd.read_sql(sql, const)
df_price.style.format(format_dict)


SELECT * FROM price WHERE name = "ORI" ORDER BY date DESC LIMIT 5



,name,date,price,maxp,minp,qty,opnp
0,ORI,2022-04-22,11.10,11.20,11.00,"5,840,474",11.20
1,ORI,2022-04-21,11.20,11.30,11.10,"1,972,933",11.20
2,ORI,2022-04-20,11.20,11.40,11.10,"6,517,836",11.30
3,ORI,2022-04-19,11.20,11.30,11.00,"5,512,016",11.10
4,ORI,2022-04-18,11.10,11.10,10.90,"7,103,264",11.00


In [8]:
sql = '''
SELECT name
FROM buy 
WHERE active = 1
ORDER BY name'''
df_price = pd.read_sql(sql, const)

names = df_price.name.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))

sql = """
SELECT name, price AS price_d 
FROM price 
WHERE date = '%s' AND name IN (%s)
ORDER BY name, date"""
sql = sql % (today, in_p)
print(sql)

df_today = pd.read_sql(sql, const)
df_today.shape[0]


SELECT name, price AS price_d 
FROM price 
WHERE date = '2022-04-22' AND name IN ('ASP', 'BCH', 'DIF', 'DOHOME', 'HREIT', 'IP', 'JASIF', 'KCE', 'MAKRO', 'MCS', 'NER', 'ORI', 'PTT', 'PTTGC', 'RATCH', 'RCL', 'RJH', 'SCC', 'SCCC', 'SENA', 'SPALI', 'STA', 'SYNEX', 'TISCO', 'TMT', 'TPIPL', 'WHART')
ORDER BY name, date


27

In [9]:
sql = """
SELECT name, price AS price_w
FROM price 
WHERE date = '%s' AND name IN (%s) 
ORDER BY name"""
sql = sql % (wkago, in_p)
print(sql)

df_wkago = pd.read_sql(sql, const)
df_wkago.shape[0]


SELECT name, price AS price_w
FROM price 
WHERE date = '2022-04-12' AND name IN ('ASP', 'BCH', 'DIF', 'DOHOME', 'HREIT', 'IP', 'JASIF', 'KCE', 'MAKRO', 'MCS', 'NER', 'ORI', 'PTT', 'PTTGC', 'RATCH', 'RCL', 'RJH', 'SCC', 'SCCC', 'SENA', 'SPALI', 'STA', 'SYNEX', 'TISCO', 'TMT', 'TPIPL', 'WHART') 
ORDER BY name


27

In [10]:
trend = pd.merge(df_today, df_wkago, on='name',how='inner')
trend.shape

(27, 3)

In [11]:
def performance(vals):
    price_d, price_w = vals
    if (price_d > price_w):
        return 'Better'
    elif (price_d < price_w):
        return 'Worse'
    else:
        return 'No Change'

In [12]:
trend['percent'] = (trend.price_d-trend.price_w)/trend.price_w * 100

In [13]:
trend

,name,price_d,price_w,percent
0,ASP,3.38,3.40,-0.59
1,BCH,22.60,22.20,1.80
2,DIF,14.20,14.10,0.71
3,DOHOME,21.40,21.40,0.00
4,HREIT,8.60,8.65,-0.58
5,IP,19.90,19.70,1.02
6,JASIF,11.00,10.90,0.92
7,KCE,63.25,61.25,3.27
8,MAKRO,38.75,39.00,-0.64
9,MCS,11.90,13.00,-8.46


In [14]:
trend["perform"] = trend[["price_d","price_w"]].apply(performance, axis=1)

In [15]:
trend.sort_values(by=['percent'],ascending=[True]).style.format(format_dict)

,name,price_d,price_w,percent,perform
24,TMT,10.30,11.30,-8.85%,Worse
9,MCS,11.90,13.00,-8.46%,Worse
10,NER,6.80,7.15,-4.90%,Worse
26,WHART,11.10,11.40,-2.63%,Worse
19,SENA,4.48,4.56,-1.75%,Worse
21,STA,25.75,26.00,-0.96%,Worse
22,SYNEX,24.80,25.00,-0.80%,Worse
18,SCCC,152.00,153.00,-0.65%,Worse
8,MAKRO,38.75,39.00,-0.64%,Worse
0,ASP,3.38,3.40,-0.59%,Worse


In [16]:
trend.perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Worse,44.44%
Better,44.44%
No Change,11.11%


In [17]:
sql = '''
SELECT name, max_price AS max, min_price AS min, pe, pbv, daily_volume AS volume, beta, market
FROM stocks
'''
my_stocks = pd.read_sql(sql, conmy)
my_stocks.shape

(359, 8)

In [18]:
filters = [
   (my_stocks.market.str.contains('SET50')),
   (my_stocks.market.str.contains('SET100')),
   (my_stocks.market.str.contains('mai'))]
values = ['SET50','SET100','mai']
my_stocks["mrkt"] = np.select(filters, values, default='SET999')

In [19]:
trend2 = pd.merge(trend, my_stocks, on='name', how='inner')
trend2.sort_values(['percent'],ascending=[True]).shape

(27, 13)

In [20]:
set50 = trend2.mrkt.str.contains('SET50') 
flt_set50 = trend2[set50]
flt_set50[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_d,price_w,percent,perform,mrkt
17,SCC,362.00,363.00,-0.28%,Worse,SET50
13,PTTGC,49.50,49.25,0.51%,Better,SET50
12,PTT,37.25,37.00,0.68%,Better,SET50
14,RATCH,44.75,44.25,1.13%,Better,SET50
23,TISCO,101.50,99.75,1.75%,Better,SET50
7,KCE,63.25,61.25,3.27%,Better,SET50


In [21]:
flt_set50.perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Better,83.33%
Worse,16.67%


In [22]:
set100 = trend2.mrkt.str.contains('SET100')
flt_set100 = trend2[set100]
flt_set100[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_d,price_w,percent,perform,mrkt
21,STA,25.75,26.00,-0.96%,Worse,SET100
22,SYNEX,24.80,25.00,-0.80%,Worse,SET100
3,DOHOME,21.40,21.40,0.00%,No Change,SET100
11,ORI,11.10,11.10,0.00%,No Change,SET100
20,SPALI,22.60,22.50,0.44%,Better,SET100
1,BCH,22.60,22.20,1.80%,Better,SET100
15,RCL,45.25,42.50,6.47%,Better,SET100


In [23]:
flt_set100[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Better,42.86%
No Change,28.57%
Worse,28.57%


In [24]:
set999 = trend2.mrkt.str.contains('SET999')
flt_set999 = trend2[set999]
flt_set999[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_d,price_w,percent,perform,mrkt
24,TMT,10.30,11.30,-8.85%,Worse,SET999
9,MCS,11.90,13.00,-8.46%,Worse,SET999
10,NER,6.80,7.15,-4.90%,Worse,SET999
26,WHART,11.10,11.40,-2.63%,Worse,SET999
19,SENA,4.48,4.56,-1.75%,Worse,SET999
18,SCCC,152.00,153.00,-0.65%,Worse,SET999
8,MAKRO,38.75,39.00,-0.64%,Worse,SET999
0,ASP,3.38,3.40,-0.59%,Worse,SET999
4,HREIT,8.60,8.65,-0.58%,Worse,SET999
25,TPIPL,1.54,1.54,0.00%,No Change,SET999


In [25]:
flt_set999[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Worse,69.23%
Better,23.08%
No Change,7.69%


In [26]:
setmai = trend2.mrkt.str.contains('mai')
flt_setmai = trend2[setmai]
flt_setmai[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_d,price_w,percent,perform,mrkt
5,IP,19.90,19.70,1.02%,Better,mai


In [27]:
flt_setmai[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Better,100.00%
